In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import tensorflow as tf

/Users/Ryan/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import InputLayer, Input
from tensorflow.python.keras.layers import Reshape, MaxPooling2D
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from keras.utils import to_categorical

Using TensorFlow backend.


In [4]:
# # Code to read csv file into colaboratory:
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google import colab
# from oauth2client.client import GoogleCredentials

# # 1. Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# #2. Get the file
# downloaded = drive.CreateFile({'id':'1meytppCXQ0WRmngIB5_ORuOWsusRHBch'}) # replace the id with id of file you want to access
# downloaded.GetContentFile('handwritten.csv')  

Will implement [conv-relu-pool]xN -> [affine]xM -> [softmax or SVM] and write script to print accuracy of models on test and train data

# Preprocess the Data

In [5]:
data = pd.read_csv("./data/handwritten_data_785.csv", encoding = "utf8")

In [6]:
np.random.seed(0)
data = data.values
np.random.shuffle(data)
X, y = data[:,1:], data[:,0]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Data Dimensions

In [8]:
print(X_train.shape)
print(X_test.shape)

#images are 28x28
img_size = 28
img_size_flat = img_size * img_size
img_shape = (img_size, img_size)

# Tuple with height, width and depth used to reshape arrays.
# This is used for reshaping in Keras.
img_shape_full = (img_size, img_size, 1)

num_channels = 1
num_classes = 26

X_train = X_train.reshape(X_train.shape[0], img_size, img_size, 1)
X_test = X_test.reshape(X_test.shape[0], img_size, img_size, 1)

(297629, 784)
(74408, 784)


In [9]:
mean_image = np.mean(X_train, axis=0).astype(np.int64)
X_train = (X_train - mean_image)/255
X_test = (X_test - mean_image)/255

In [10]:
train_Y_one_hot = to_categorical(y_train)
test_Y_one_hot = to_categorical(y_test)

# Test: One layer of each ([conv-relu-pool] -> [affine]-> [softmax or SVM])

Remember, deeper networks is always better, at the cost of more data and increased complexity of learning.
Minibatch size is usually set of few hundreds. 
You should initially use fewer filters and gradually increase and monitor the error rate to see how it is varying.
Very small filter sizes will capture very fine details of the image. On the other hand having a bigger filter size 
will leave out minute details in the image.
https://www.quora.com/How-can-I-decide-the-kernel-size-output-maps-and-layers-of-CNN

# Keras

In [18]:
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.layers import LeakyReLU
from tensorflow.python.keras import initializers

optimizer = Adam(lr=1e-3)

In [24]:
model = Sequential()

'''
initializer: he_normal
A more recent paper on this topic, Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet Classification by He et al., 
derives an initialization specifically for ReLU neurons, reaching the conclusion that the variance of neurons 
in the network should be 2.0/n. This gives the initialization w = np.random.randn(n) * sqrt(2.0/n), 
and is the current recommendation for use in practice in the specific case of neural networks with ReLU neurons.
'''
#convolutional layer
model.add(Conv2D(kernel_size=7, strides=1, filters=16, padding='same',
                 activation='linear', name='layer_conv1', input_shape=img_shape_full))
#reLU activation
model.add(LeakyReLU(alpha=0.1))
#Pooling Layer
model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))

model.add(Flatten())
#Fully-Connected Layer with 128 outputs
model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha=0.1))
#Fully-Connected Layer with Softmax
model.add(Dense(num_classes, activation='softmax'))

'''
For multiclass classification problems like MNIST, cross entropy is typically used as the loss metric
'''
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_conv1 (Conv2D)         (None, 28, 28, 16)        800       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 28, 28, 16)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               401536    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 26)                3354      
Total para

In [25]:
model.fit(x=X_train,
          y=train_Y_one_hot,
          epochs=10, batch_size=128,verbose=1, validation_data=(X_test, test_Y_one_hot))

Train on 297629 samples, validate on 74408 samples
Epoch 1/10
297629/297629 [==============================] - 192s 646us/step - loss: 0.1367 - acc: 0.9628 - val_loss: 0.0831 - val_acc: 0.9780
Epoch 2/10
297629/297629 [==============================] - 174s 584us/step - loss: 0.0669 - acc: 0.9818 - val_loss: 0.0680 - val_acc: 0.9818
Epoch 3/10
297629/297629 [==============================] - 173s 581us/step - loss: 0.0516 - acc: 0.9860 - val_loss: 0.0595 - val_acc: 0.9842
Epoch 4/10
297629/297629 [==============================] - 175s 589us/step - loss: 0.0410 - acc: 0.9888 - val_loss: 0.0538 - val_acc: 0.9860
Epoch 5/10
297629/297629 [==============================] - 175s 589us/step - loss: 0.0335 - acc: 0.9907 - val_loss: 0.0522 - val_acc: 0.9873
Epoch 6/10
297629/297629 [==============================] - 174s 585us/step - loss: 0.0276 - acc: 0.9924 - val_loss: 0.0489 - val_acc: 0.9881
Epoch 7/10
297629/297629 [==============================] - 174s 585us/step - loss: 0.0237 - acc:

<strong>without he_normal weights initialization and 0.01 biases initialization:</strong>

Train on 297629 samples, validate on 74408 samples

Epoch 1/1
loss: 0.1502 - acc: 0.9591 

val_loss: 0.0927 - val_acc: 0.9762

<strong>using he_normal weight initialization and 0.01 biases initialization actually decreased accuracy by 0.0014</strong>

Train on 297629 samples, validate on 74408 samples

Epoch 1/1

loss: 0.1835 - acc: 0.9494 - 

val_loss: 0.0963 - val_acc: 0.9748